# Business Case 4

##  ManyGiftsUK: Online Retailer Recommender System

**Group members:**
- Lorenzo Pigozzi	--- m20200745
- Nguyen Huy Phuc	--- m20200566
- Ema Mandura	  --- m20200647
- Xavier Goncalves  --- m20201090

**Expected outcomes**
- Explore the data and build models to answer the problems:
 - Recommender system: appears in the website homepage and offers a wide range of relevant products to each user;
 - Cold start: offer relevant products to new customers;
- Implement adequate evaluation strategies and select an appropriate quality measure
- In the deployment phase, elaborate on the challenges and recommendations in implementing the recommender system


<a class="anchor" id="0.1"></a>
# **Table of Contents**

1.	[Importing data and libraries](#1)   
2.	[Exploratory data analysis (EDA)](#2)       

# 1. Importing data and libraries <a class="anchor" id="1"></a>

In [2]:
# Import packages
import pandas as pd
import numpy as np
import datetime as dt
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
import matplotlib.pyplot as plt
import networkx as nx
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from os import path
from wordcloud import WordCloud

In [3]:
# Read csv file
df = pd.read_csv('datasets\events_example.csv')

# 2. Exploratory Data Analysis <a class="anchor" id="2"></a>

In [4]:
df.head()

,timestamp,visitorid,event,itemid,transactionid
0,1433221332117,257597,view,355908,NaN
1,1433224214164,992329,view,248676,NaN
2,1433221999827,111016,view,318965,NaN
3,1433221955914,483717,view,253185,NaN
4,1433221337106,951259,view,367447,NaN


In [6]:
df.shape

(2756101, 5)

In [8]:
df.isna().sum()

timestamp              0
visitorid              0
event                  0
itemid                 0
transactionid    2733644
dtype: int64

In [9]:
# Describe dataset
df.describe(include='all')

,timestamp,visitorid,event,itemid,transactionid
count,2.756101e+06,2.756101e+06,2756101,2.756101e+06,22457.000000
unique,NaN,NaN,3,NaN,NaN
top,NaN,NaN,view,NaN,NaN
freq,NaN,NaN,2664312,NaN,NaN
mean,1.436424e+12,7.019229e+05,NaN,2.349225e+05,8826.497796
std,3.366312e+09,4.056875e+05,NaN,1.341954e+05,5098.996290
min,1.430622e+12,0.000000e+00,NaN,3.000000e+00,0.000000
25%,1.433478e+12,3.505660e+05,NaN,1.181200e+05,4411.000000
50%,1.436453e+12,7.020600e+05,NaN,2.360670e+05,8813.000000
75%,1.439225e+12,1.053437e+06,NaN,3.507150e+05,13224.000000


In [10]:
# Value counts of event
df['event'].value_counts()

view           2664312
addtocart        69332
transaction      22457
Name: event, dtype: int64